In [ ]:
# ! pip install nbformat>=4.2.0

In [1]:
import os, sys
from datetime import datetime as dt, timedelta as td
from time import sleep
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


absolute_parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(absolute_parent)

from tdf_utility.trading.nse_api import NSE_API

In [2]:
company_ticker = "BHARTIARTL"
# from_dt = "15-01-2025"
# to_dt = "12-01-2026"

# print(company_ticker, from_dt, to_dt)

In [ ]:
top_10_stock_url = "https://www.nseindia.com/api/NextApi/apiClient/GetQuoteApi?functionName=getTopTenStock"
nifty_index = "https://www.nseindia.com/api/NextApi/apiClient?functionName=getIndexData&&type=All"

In [ ]:
def company_viz_df(company_ticker, from_dt:str=None, to_dt:str=None):
    company_details = "https://www.nseindia.com/api/NextApi/apiClient/GetQuoteApi?functionName=getSymbolData&marketType=N&series=EQ&symbol=BHARTIARTL"

    company_1y_chart_data = "https://www.nseindia.com/api/NextApi/apiClient/GetQuoteApi?functionName=getSymbolChartData&symbol=BHARTIARTLEQN&days=1Y"
    
    
    
    nse_api = NSE_API()
    df_list = list()

    company_name_url = f"api/NextApi/apiClient/GetQuoteApi?functionName=getSymbolName&symbol={company_ticker}"
    company_name = nse_api._get_data(company_name_url).get("companyName")

    company_yearwise_url = f"api/NextApi/apiClient/GetQuoteApi?functionName=getYearwiseData&symbol={company_ticker}QN"
    # company_yearwise_data = 
    

    if from_dt and to_dt:
        to_dt = dt.strptime(to_dt, '%d-%m-%Y')
        from_dt = dt.strptime(from_dt, '%d-%m-%Y')
        company_corp_action_url = f"api/NextApi/apiClient/GetQuoteApi?functionName=getCorpAction&symbol={company_ticker}&type=W&marketApiType=equities&ex_from_date={from_dt}&ex_to_date={to_dt}"
    else:
        to_dt = dt.now()
        from_dt = to_dt - td(days=365)
        company_corp_action_url = f"api/NextApi/apiClient/GetQuoteApi?functionName=getCorpAction&symbol={company_ticker}&type=W&marketApiType=equities&ex_from_date={from_dt.strftime("%d-%m-%Y")}&ex_to_date={to_dt.strftime("%d-%m-%Y")}"

    while from_dt < to_dt:
        from_dt_30 =  (to_dt - td(days=30))
        company_historical_tradedata_url = f'api/NextApi/apiClient/GetQuoteApi?functionName=getHistoricalTradeData&symbol={company_ticker}&series=EQ&fromDate={from_dt_30.strftime("%d-%m-%Y")}&toDate={to_dt.strftime("%d-%m-%Y")}'
        # print(from_dt_30, to_dt)
        # print(dynamic_url)
        company_trade_data = nse_api._get_data(company_historical_tradedata_url)
        df_list.append(pd.DataFrame(company_trade_data))
        # print(from_dt_30, to_dt)
        to_dt = (from_dt_30 - td(days=1))
        
    company_trade_data_df = pd.concat(df_list[::-1], ignore_index=True)
    company_trade_data_df["Date"] = pd.to_datetime(company_trade_data_df["mtimestamp"], format='%d-%b-%Y')
    company_trade_data_df.rename(columns={"chSymbol": "Symbol", 
                                          "chOpeningPrice": "Open",
                                          "chTradeHighPrice": "High",
                                          "chTradeLowPrice": "Low",
                                          "chClosingPrice": "Close",
                                          "chPreviousClsPrice": "Prev Close",
                                          "chLastTradedPrice": "LTP",
                                          "vwap": "VWAP",
                                          "chTotTradedQty": "Volume",	
                                          "chTotTradedVal": "Turnover",	
                                          "chTotalTrades": "Transactions",
                                          "ch52WeekHighPrice": "52 Week High",
                                          "ch52WeekLowPrice": "52 Week Low"}, inplace=True)
    company_trade_data_df = company_trade_data_df.loc[:, ["Date", "Symbol", "Open", "High", "Low", "Close", "Prev Close", "LTP", "VWAP", "Volume", "Turnover", "Transactions", "52 Week High", "52 Week Low"]]
    company_trade_data_df = company_trade_data_df.sort_values(by='Date')
    company_trade_data_df.reset_index(drop=True, inplace=True)
    company_trade_data_df['7_DMA'] = company_trade_data_df['Close'].rolling(window=7).mean()
    company_trade_data_df['21_DMA'] = company_trade_data_df['Close'].rolling(window=21).mean()
    
    # company_df = pd.DataFrame(nse_api._get_data(company_url))
    company_corp_action_df = pd.DataFrame(nse_api._get_data(company_corp_action_url))
    return company_name, company_trade_data_df, company_corp_action_df


In [5]:
# company_trade_data_df, company_corp_action_df = company_viz_df(company_ticker=company_ticker, from_dt=from_dt, to_dt=to_dt)
company_name, company_trade_data_df, company_corp_action_df = company_viz_df(company_ticker=company_ticker)

error from NSE_API.get_data(): 404 Client Error: Not Found for url: https://www.nseindia.com/https://www.nseindia.com/api/NextApi/apiClient/GetQuoteApi?functionName=getSymbolName&symbol=BHARTIARTL


In [ ]:
fig_company = make_subplots(specs=[[{"secondary_y": True}]])

# Add Trace 1: Nifty 50 (Left Axis)
fig_company.add_trace(
    go.Scatter(
        x=company_trade_data_df['Date'], 
        y=company_trade_data_df['Close'], 
        name="close",
        line=dict(color="#FFFFFF", width=2)
    ),
    secondary_y=False, # Assign to primary (left) axis
)

fig_company.add_trace(
    go.Scatter(
        x=company_trade_data_df['Date'], 
        y=company_trade_data_df['High'], 
        name="high",
        line=dict(color="#FF0000", width=2)
    ),
    secondary_y=False, # Assign to primary (left) axis
)

fig_company.add_trace(
    go.Scatter(
        x=company_trade_data_df['Date'], 
        y=company_trade_data_df['Low'], 
        name="low",
        line=dict(color='#00C805', width=2)
    ),
    secondary_y=False, # Assign to primary (left) axis
)

# fig_nifty_indiavix.update_yaxes(title_text="<b>Nifty Price</b>", secondary_y=False)
# fig_nifty_indiavix.update_yaxes(title_text="<b>VIX Level</b>", secondary_y=True)

fig_company.update_layout(
    title=f"Nifty50 and India VIX Trend",
    # xaxis_title="Date",
    yaxis_title="Nifty50",
    template="plotly_dark", # Use 'plotly_white' for light theme
    hovermode="x unified",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig_company

In [ ]:
fig_company = go.Figure(data=[go.Candlestick(x=company_trade_data_df['Date'],
                open=company_trade_data_df['Open'],
                high=company_trade_data_df['High'],
                low=company_trade_data_df['Low'],
                close=company_trade_data_df['Close'])])

fig_company.show()

Exception: In order to reference traces by row and column, you must first use plotly.tools.make_subplots to create the figure with a subplot grid.

In [ ]:


# Assuming 'company_trade_data_df' is your DataFrame and it is sorted by Date.
# 1. CALCULATE THE MOVING AVERAGES
# We use the 'rolling' window function on the Close price.


# 2. CREATE THE BASE CANDLESTICK CHART
fig_company = go.Figure(data=[go.Candlestick(
    x=company_trade_data_df['Date'],
    open=company_trade_data_df['Open'],
    high=company_trade_data_df['High'],
    low=company_trade_data_df['Low'],
    close=company_trade_data_df['Close'],
    name="Price" # Legend name
)])

# 3. ADD THE 7 DMA LINE (Standard Color: Blue or Green)
fig_company.add_trace(go.Scatter(
    x=company_trade_data_df['Date'],
    y=company_trade_data_df['7_DMA'],
    mode='lines',
    name='7 DMA',
    line=dict(color='blue', width=1.5)
))

# 4. ADD THE 21 DMA LINE (Standard Color: Red or Black)
fig_company.add_trace(go.Scatter(
    x=company_trade_data_df['Date'],
    y=company_trade_data_df['21_DMA'],
    mode='lines',
    name='21 DMA',
    line=dict(color='black', width=1.5)
))

# 5. ADD THE VWAP Line
fig_company.add_trace(
    go.Scatter(
        x=company_trade_data_df['Date'], 
        y=company_trade_data_df['VWAP'], 
        mode='lines',
        name="Vwap",
        line=dict(color="#A6FF00", width=1.2)
    )
)

# Optional: Clean up the layout for better analysis
fig_company.update_layout(
    title="Stock Price with 7 & 21 DMA",
    xaxis_title="Date",
    yaxis_title="Price",
    xaxis_rangeslider_visible=True # Often cleaner without the bottom slider
)

# Display in Streamlit
# st.plotly_chart(fig_company, use_container_width=True)

In [1]:
df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(df, x="year", y="lifeExp", color='country')
fig.show()

NameError: name 'px' is not defined

In [19]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
60,Australia,Oceania,1952,69.120,8691212,10039.59564,AUS,36
61,Australia,Oceania,1957,70.330,9712569,10949.64959,AUS,36
62,Australia,Oceania,1962,70.930,10794968,12217.22686,AUS,36
63,Australia,Oceania,1967,71.100,11872264,14526.12465,AUS,36
64,Australia,Oceania,1972,71.930,13177000,16788.62948,AUS,36
65,Australia,Oceania,1977,73.490,14074100,18334.19751,AUS,36
66,Australia,Oceania,1982,74.740,15184200,19477.00928,AUS,36
67,Australia,Oceania,1987,76.320,16257249,21888.88903,AUS,36
68,Australia,Oceania,1992,77.560,17481977,23424.76683,AUS,36
69,Australia,Oceania,1997,78.830,18565243,26997.93657,AUS,36


In [14]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- 1. Data Simulation (Matching your provided start/end points) ---
def generate_market_data():
    dates = pd.date_range(start="2024-12-30", periods=250, freq='B') # Business days
    
    # Linear interpolation + Noise for realistic look
    nifty = np.linspace(23644, 25938, len(dates)) + np.random.normal(0, 150, len(dates))
    vix = np.linspace(13.97, 9.68, len(dates)) + np.random.normal(0, 0.5, len(dates))
    gold = np.linspace(75953, 134158, len(dates)) + np.random.normal(0, 500, len(dates))
    
    return pd.DataFrame({'Date': dates, 'Nifty 50': nifty, 'India VIX': vix, 'Gold': gold})

df = generate_market_data()

# --- 2. Streamlit Setup ---


# --- 3. Build Stacked Subplots ---
# rows=3, cols=1 creates a vertical stack
fig = make_subplots(
    rows=3, cols=1, 
    shared_xaxes=True, # vital for comparing timeframes
    vertical_spacing=0.05,
    subplot_titles=("Nifty 50 (Equity)", "India VIX (Volatility)", "Gold 10gm (Commodity)"),
    specs=[[{"secondary_y": False}], [{"secondary_y": False}], [{"secondary_y": False}]]
)

# Row 1: Nifty 50
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['Nifty 50'],
    name="Nifty 50", mode='lines',
    line=dict(color='#00E396', width=2) # Green
), row=1, col=1)

# Row 2: India VIX (Inverse indicator, often red/orange)
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['India VIX'],
    name="India VIX", mode='lines',
    line=dict(color='#FF4560', width=2) # Red
), row=2, col=1)

# Row 3: Gold (Safe haven, often yellow/gold)
fig.add_trace(go.Scatter(
    x=df['Date'], y=df['Gold'],
    name="Gold", mode='lines',
    line=dict(color='#FEB019', width=2) # Gold color
), row=3, col=1)

# --- 4. Professional Styling ---
fig.update_layout(
    height=800, # Taller chart to accommodate 3 panels
    template="plotly_dark",
    hovermode="x unified", # Shows all 3 values in one tooltip when hovering a date
    showlegend=False, # Hide legend to save space (titles are sufficient)
    margin=dict(t=50, b=50, l=50, r=50)
)

# Add Range Slider only to the bottom plot, but it controls all
fig.update_xaxes(matches='x') 


In [20]:

import pandas as pd

# 1. Setup Dummy Data (Matching your previous structure)
data = {
    "Date": ["2025-12-29", "2025-12-30"],
    "Nifty 50": [25942.10, 25938.85],  # Price dropped slightly
    "India VIX": [9.72, 9.68]          # Volatility dropped slightly
}
df = pd.DataFrame(data)

# 2. Calculation Logic
# Get the latest data (Last row)
latest_data = df.iloc[-1]
# Get the previous data (Second to last row)
prev_data = df.iloc[-2]

# Calculate Changes (Deltas)
nifty_change = latest_data["Nifty 50"] - prev_data["Nifty 50"]
vix_change = latest_data["India VIX"] - prev_data["India VIX"]

# 3. Create Columns
col1, col2 = st.columns(2)

# 4. Render Metrics
# f"{value:.2f}" formats the number to 2 decimal places

print(f"{latest_data['Nifty 50']:.2f}", f"{nifty_change:.2f}")

print(f"{latest_data['India VIX']:.2f}", f"{vix_change:.2f}")

2025-12-30 20:19:34.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:19:34.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-30 20:19:34.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


25938.85 -3.25
9.68 -0.04


In [ ]:
# Ray Dalio: bridgewater associates: Biggest Hedge Fund in the World
# warren buffett: berkshire hathaway: multinational holding company that owns entire businesses (like GEICO, Dairy Queen) and significant stakes in public companies
# charlie munger: berkshire hathaway vice chairman:

# Gary Brecka: can predict who will die when and expert on this. Top insurer company hire him.

# 100 year old banner chart

# Inflation is ok?
# Interest Rate is stable or not?

1. Rapid Api
2. Apify
3. apidog
4. https://developer.schwab.com/user-guides/get-started/introduction
